## Example of STM in use
This is different than poliblogs.ipynb because we are demonstrating use of metadata

In [1]:
import numpy as np
import pandas as pd
from patsy import dmatrix
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from py_stm.stm import StmModel
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models

In [2]:
poliblogs = pd.read_csv("test_data/poliblogs2008.csv", )
poliblogs = poliblogs.loc[:, ~poliblogs.columns.str.contains('^Unnamed')]

poliblogs.head()

,documents,docname,rating,day,blog
0,"After a week of false statements, lies, and di...",at0800300_1.text,Conservative,3,at
1,I honestly don't know how either party's caucu...,at0800300_2.text,Conservative,3,at
2,While we stand in awe of the willingness of ou...,at0800300_3.text,Conservative,3,at
3,These pages recently said goodbye to global wa...,at0800300_4.text,Conservative,3,at
4,A US report shows how the enemy controlled the...,at0800300_5.text,Conservative,3,at


In [3]:
print(f"There are {len(poliblogs)} many documents in the poliblogs dataset")

There are 13246 many documents in the poliblogs dataset


In [4]:
nltk.download('stopwords')  # Download the stopwords

[nltk_data] Downloading package stopwords to C:\Users\Tyler
[nltk_data]     Holston\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def preprocess_text(text, min_len):
    tokens = simple_preprocess(text, deacc=True, min_len=min_len)  # deacc=True removes punctuation
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    return filtered_tokens


In [6]:
# Split the text and metadata for training
train_text, test_text, train_metadata, test_metadata = train_test_split(
    poliblogs.documents, poliblogs[['rating', 'day', 'blog']], test_size=0.2, random_state=42
)

# Apply the preprocessing function to the text data
processed_train_text = [preprocess_text(text, min_len=3) for text in train_text]
processed_test_text = [preprocess_text(text, min_len=3) for text in test_text]

# Create the training dictionary
dictionary = Dictionary(processed_train_text)

# Filter extremes (remove tokens that appear in less than 10 documents, or more than 50% of the documents)
dictionary.filter_extremes(no_below=10, no_above=0.5)

# Create the training corpus (bag of words representation)
corpus_train = [dictionary.doc2bow(text) for text in processed_train_text]

In [25]:
# A user could precompute the prevalence themselves like so
from patsy import dmatrix

prevalence = dmatrix("~rating+cr(day, df=3)", data=train_metadata, return_type='dataframe')

In [ ]:
# Train the STM model
num_topics = 5  # Define the number of topics you want to extract
stm = StmModel(corpus_train, num_topics=num_topics, id2word=dictionary, prevalence=prevalence, passes=2, random_state=420) # intended use 1. prevalence matrix precomputed
stm = StmModel(corpus_train, num_topics=num_topics, id2word=dictionary, metadata=train_metadata, prevalence="~rating+cr(day, df=3)", passes=2, random_state=420) # intended use 2. metadata dataframe with prevalence formula
stm = StmModel(corpus_train, num_topics=num_topics, id2word=dictionary, metadata=train_metadata, content="~blog", passes=2, random_state=420) # intended use 3. metadata dataframe and content formula